In [7]:
import pandas as pd
from datasets import Datasets

pth = "../"
data = Datasets(pth + "X_train.csv", pth + "X_test.csv", pth + "X_val.csv",
                pth + "y_train.csv", pth + "y_test.csv", pth + "y_val.csv")

# def feature_select_pearson(train, test):
print('Filter method to select top 200 features...')
train = data.X_train
test = data.X_test
target = "CRASH_SEVERITY"
train[target] = data.y_train[target]
test[target] = data.y_test[target]
features = train.columns.tolist()
# features.remove("CRASH_ID")
features.remove(target)
featureSelect = features[:]


# remove features with a missing value ratio greater than 0.99
for feature in features:
    if train[feature].isnull().sum() / train.shape[0] >= 0.99:
        featureSelect.remove(feature)

# calculate the pearson correlation
corr = []
for feature in featureSelect:
    corr.append(abs(train[[feature, target]].fillna(0).corr().values[0][1]))

# get top 200 features with the highest correlation
se = pd.Series(corr, index=featureSelect).sort_values(ascending=False)
feature_select = se[:200].index.tolist()
print('done')
print(train[feature_select + [target]], test[feature_select + [target]])
# return train[feature_select + [target]], test[feature_select + [target]]

Filter method to select top 300 features...


/tmp/ipykernel_1833/2539633209.py:29: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  corr.append(abs(train[[feature, target]].fillna(0).corr().values[0][1]))


IndexError: index 1 is out of bounds for axis 0 with size 1